This notebook calculates contraints on starspot contamination for GJ 1132, based on observations from Hubble/WFC3 and ground-based telescopes, as presented in Libby-Roberts et al. (2022). It was written by Zach Berta-Thompson in spring 2021, and revised in spring 2022.

We'll start by importing tools from the [`starspot-backlights`](https://github.com/zkbt/starspot-backlights) package, which contains the basic modeling framework and tools. You can install them via `pip install starspot-backlights`. The current version is `0.0.2`.

In [ ]:
from starspotbacklights import *

First, let's set up the data we will use as a constraint. We'll make a dictionary, called `all_data`, and populate it with a few different kinds of data. 

In [ ]:
all_data = {}

# Data += Out-of-Transit Modulations

First, we'll add some information about the amplitude of the out-of-transit modulations cause by the rotation of the star. This is expressed as, approximately, the semi-amplitude of the flux variations in one or more photometric bandpasses (which may be custom, effectively spectroscopic, bandpasses).

In the case of GJ1132, the semi-amplitude (= half the peak-to-peak ampltiude) seems to be about 1% by visual inspection of its out-of-transit light curve. We'll give it an approximate error bar of about 0.2%.

In [ ]:
# add a data point for the amplitude of out of transit modulations
oot = pd.DataFrame({'amplitude':0.01, 
                    'filter':MEarth(),
                    'amplitude-error':0.002}, index=[0])

oot

In [ ]:
# add this out-of-transit table to the data being fit
all_data['oot'] = oot

# Data += The Overall Spectrum of the Star
Next, let's make sure that the integrated flux from our multi-component stellar surface adds up to be something that looks like the real star. For example, we want to exclude models were both the "unspotted" and the "spotted" photosphere have temperatures that are both way warmer or way colder than the actual temperature of the star. 

The semi-kludgy way to do this is to make sure that the integrated surface flux of the star works out to have something near the effective temperature of the star as determined from its bolometric flux. The more precise way to do this would likely be to try to reproduce the actual absolute spectrum of the star for at least some wavelengths -- let's consider that TBD!

In the case of GJ1132, the effective temperature of the star (from Berta-Thompson + Bonfils + ...) is $3270\pm140$K. We'll use that.

In [ ]:
# put in one data point for the effective temperature of the star
teff = pd.DataFrame({'teff':3270, 
                     'teff-error':140}, index=[0])
teff

In [ ]:
# add this average stellar spectrum constraint to the data being fit
all_data['teff'] = teff

# Data += The Observed Transmission Spectrum

Next, let's include some observations of the transit depth as a function of wavelength. Those constrain a unique combination of starspot parameters and are particularly sensitive to the total spot covering fraction $f$.

In the case of GJ1132, let's start with including the depths from Jessica Libby-Roberts' WFC3 paper. 

In [ ]:
# read Jessica Libby-Roberts' table of transit depths from WFC3
wfc3_depths = pd.read_csv('gj1132b-depth-data/hst_depths.tex', delimiter='\t',
                   names=['wavelength', 'depth', 'depth-error'])

# create some filters associated with each wavelength bin
dw = np.round(np.median(np.diff(wfc3_depths['wavelength'])), decimals=5)
wfc3_depths['filter'] = [Filter(w*1000, dw*1000) for w in wfc3_depths['wavelength']]

wfc3_depths

In [ ]:
# add to the data constraints
all_data['depth'] = wfc3_depths

We will also add in Hannah Diamond-Lowe's [LDSS3C transit depths](https://ui.adsabs.harvard.edu/abs/2018AJ....156...42D/abstract).

In [ ]:
# read Hannah Diamond-Lowe's table from LDSSC3
ldss3c = pd.read_csv('gj1132b-depth-data/diamondlowe_depths.txt', delimiter=',',
                   names=['wavelength', 'depth', 'depth-error'])

# create some filters associated with each wavelength bin
dw = np.round(np.median(np.diff(ldss3c['wavelength'])), decimals=5)
ldss3c['filter'] = [Filter(w*1000, dw*1000) for w in ldss3c['wavelength']]

ldss3c

Since we have some uncertainties about the absolute level of the transit depths for these data, we will include them as `relative-depth`. This is a kludge that we added to the code; these depths will be allowed to move up and down as a group went fitting. (If you'll have more than two different depth datasets that will all need to shift relative to each other, you'll need to modify the `starspot-backlights` definitions. Sorry!)

In [ ]:
all_data['relative-depth'] = ldss3c

In [ ]:
for k in all_data:
    print(f'There are {len(all_data[k]):2} "{k}" data points.')

# Infer Some Starspot Parameters

Now that we've defined all that useful data, let's run a little MCMC to explore how well we can reproduce the provided data with a simple model of a spotted star. This particular model includes a gentle prior on how to connect the time-variable component of the spot-covering fraction $\Delta f$ to overall average starspot-covering fraction $f$. It's set in a kind-of-kludgy Poisson fashion by the fractional covering area of one starspot $f_1$. 

We'll use MCMC to sample from the probability distributions

In [ ]:
# sample with an increasing number of steps 
for N in [100, 1000, 10000]:
    # sample using different subsets of the data
    for to_include in [['oot', 'teff', 'depth', 'relative-depth'],
                       ['oot', 'teff', 'depth']]:

        # create a unique label for this subset of data
        label = '+'.join(to_include)
        data_to_use = {k:all_data[k] for k in to_include}
        print(f'{label} with {N} steps\n' + '-'*10)

        # create a backlight object, which we'll use to sample
        b = Backlight(data_to_use, 
                      include_poisson=True, 
                      max_temperature_offset=0.25, 
                      stellar_radius=0.2105, 
                      planet_radius=1.130,
                      logg=5.0) 

        # create (or load already generated) MCMC samples
        b.sample(N, N)

        # make a multipanel plot of the data and model parameters
        b.plot_everything()
        b.legend_amplitude.get_texts()[0].set_text('MEarth')
        b.legend_depth.get_texts()[0].set_text('WFC3')
        if 'relative-depth' in to_include:
            b.legend_depth.get_texts()[1].set_text('Magellan')
        plt.savefig(f'GJ1132b-starspots-everything-{label}-{N}.pdf');

Next, let's ask the question of whether it's possible to hide the signal of a H-rich atmosphere by canceling out its transmission spectrum with an unfortunate arrangement of spots. To do this, we'll make a fake dataset with flipped a H-rich transmission spectrum injected into it. If we can model that well with spots, then spots could cancel out a real transmission spectrum. 

In [ ]:
# read a table of model transit depths
solar_model_depths = pd.read_csv('gj1132b-depth-data/100xsolar_binned.txt', names=['wavelength', 'depth'], delimiter=' ')

# make a fake set of transit depths
fake_wfc3_depths = copy.deepcopy(wfc3_depths)
simulation = np.interp(wfc3_depths['wavelength'], solar_model_depths['wavelength'], solar_model_depths['depth'])
fake_wfc3_depths['depth'] -= (simulation - np.mean(simulation))

# make a fake complete dataset, with these fake depths
faked_all_data = copy.deepcopy(all_data)
faked_all_data['depth'] = fake_wfc3_depths

In [ ]:
# sample with an increasing number of steps 
for N in [100, 1000, 10000]:
    # sample using different subsets of the data
    for to_include in [['oot', 'teff', 'depth']]:

        # create a unique label for this subset of data
        label = '+'.join(to_include)
        data_to_use = {k:faked_all_data[k] for k in to_include}
        print(f'{label} with {N} steps\n' + '-'*10)
        
        # create a backlight object, which we'll use to sample
        b = Backlight(data_to_use, 
                      include_poisson=True, 
                      max_temperature_offset=0.25, 
                      stellar_radius=0.2105, 
                      planet_radius=1.130,
                      logg=5.0,
                      label='fake100xsolar') 
        
        # create (or load already generated) MCMC samples
        b.sample(N, N)
        
        # make a multipanel plot of the data and model parameters
        b.plot_everything()
        b.legend_amplitude.get_texts()[0].set_text('MEarth')
        b.legend_depth.get_texts()[0].set_text('WFC3')
                
        plt.savefig(f'GJ1132b-starspots-everything-{label}-{N}-fake100xsolar.pdf');